In [210]:
# TensorFlow and tf.keras
import math
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras


#tf version should be 2.5 or higher
tf.__version__

'2.6.0'

In [211]:
train=pd.read_csv('sign_mnist_train.csv')
test=pd.read_csv('sign_mnist_test.csv')

In [212]:
train_images=train.values[0:,1:]

test_images=test.values[0:,1:]

In [213]:
labels = train.values[0:,0]




labels_test = test.values[0:,0]


In [214]:
train_images=train_images/255
train_images=train_images.reshape((27455,28,28,1))

test_images=test_images/255

test_images=test_images.reshape((7172, 28,28,1))


In [215]:
#setup model
model = keras.Sequential([
   
    
    keras.layers.Conv2D(kernel_size=3, filters=32, use_bias=False, padding='same', input_shape=(28, 28 , 1)),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
    
    
    keras.layers.Conv2D(kernel_size=6, filters=64, use_bias=False, padding='same', strides=2),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
   
    
    keras.layers.Conv2D(kernel_size=6, filters=64, use_bias=False, padding='same', strides=2),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
        
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
   
    
    keras.layers.Flatten(),
    

    
    keras.layers.Dense(64, use_bias=False),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(25, activation = tf.nn.softmax)
])

In [216]:
#compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [217]:
def lr_decay(epoch):
    return 0.01*math.pow(0.6, epoch)

lr_decay_callback=tf.keras.callbacks.LearningRateScheduler(lr_decay)

In [218]:
#train model
model.fit(train_images, labels, validation_split=0.1, epochs=6, batch_size=64, callbacks=[lr_decay_callback])

Epoch 1/6
387/387 [==============================] - 12s 29ms/step - loss: 1.1794 - accuracy: 0.6012 - val_loss: 0.7804 - val_accuracy: 0.7174
Epoch 2/6
387/387 [==============================] - 11s 29ms/step - loss: 0.1998 - accuracy: 0.9282 - val_loss: 0.8429 - val_accuracy: 0.7498
Epoch 3/6
387/387 [==============================] - 11s 29ms/step - loss: 0.1044 - accuracy: 0.9644 - val_loss: 0.0108 - val_accuracy: 0.9989
Epoch 4/6
387/387 [==============================] - 11s 29ms/step - loss: 0.0724 - accuracy: 0.9752 - val_loss: 2.6798e-04 - val_accuracy: 1.0000
Epoch 5/6
387/387 [==============================] - 11s 29ms/step - loss: 0.0595 - accuracy: 0.9795 - val_loss: 3.3375e-04 - val_accuracy: 1.0000
Epoch 6/6
387/387 [==============================] - 12s 30ms/step - loss: 0.0453 - accuracy: 0.9847 - val_loss: 3.2555e-05 - val_accuracy: 1.0000


In [219]:
#evaluate
test_loss, test_acc = model.evaluate(test_images,  labels_test)
print('\nTest accuracy:', test_acc)

225/225 [==============================] - 1s 5ms/step - loss: 0.3334 - accuracy: 0.9335

Test accuracy: 0.9334913492202759


In [220]:
_, baseline_model_accuracy = model.evaluate(
    test_images, labels_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9334913492202759
Saving model to:  /tmp/tmpdnbqkfc0.h5


In [226]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 6 epochs.
batch_size = 64
epochs = 6
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs


pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_3 (None, 28, 28, 32)        578       
_________________________________________________________________
prune_low_magnitude_batch_no (None, 28, 28, 32)        97        
_________________________________________________________________
prune_low_magnitude_activati (None, 28, 28, 32)        1         
_________________________________________________________________
prune_low_magnitude_max_pool (None, 14, 14, 32)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_3 (None, 7, 7, 64)          147458    
_________________________________________________________________
prune_low_magnitude_batch_no (None, 7, 7, 64)          193       
_________________________________________________________________
prune_low_magnitude_activati (None, 7, 7, 64)        

/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [227]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/6
  5/387 [..............................] - ETA: 37s - loss: 2.3894 - accuracy: 0.9219 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0272s vs `on_train_batch_end` time: 0.0471s). Check your callbacks.


387/387 [==============================] - 16s 31ms/step - loss: 2.3378 - accuracy: 0.9603 - val_loss: 2.2858 - val_accuracy: 0.9996
Epoch 2/6
387/387 [==============================] - 12s 30ms/step - loss: 2.3141 - accuracy: 0.9796 - val_loss: 2.2854 - val_accuracy: 1.0000
Epoch 3/6
387/387 [==============================] - 12s 30ms/step - loss: 2.3115 - accuracy: 0.9811 - val_loss: 2.2859 - val_accuracy: 1.0000
Epoch 4/6
387/387 [==============================] - 12s 30ms/step - loss: 2.3182 - accuracy: 0.9756 - val_loss: 2.2859 - val_accuracy: 0.9996
Epoch 5/6
387/387 [==============================] - 12s 30ms/step - loss: 2.3252 - accuracy: 0.9742 - val_loss: 2.3100 - val_accuracy: 0.9782
Epoch 6/6
387/387 [==============================] - 12s 31ms/step - loss: 2.3209 - accuracy: 0.9773 - val_loss: 2.2854 - val_accuracy: 1.0000


In [228]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, labels_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9334913492202759
Pruned test accuracy: 0.9127160906791687


In [231]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmph1jj_z7t/assets


INFO:tensorflow:Assets written to: /tmp/tmph1jj_z7t/assets


Saved pruned TFLite model to: /tmp/tmptduyh_kl.tflite


In [232]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [233]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 849709.00 bytes
Size of gzipped pruned Keras model: 273083.00 bytes
Size of gzipped pruned TFlite model: 266698.00 bytes


In [238]:



converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmpavf10_zf/assets


INFO:tensorflow:Assets written to: /tmp/tmpavf10_zf/assets


Saved quantized and pruned TFLite model to: /tmp/tmpkq6ixoyj.tflite
Size of gzipped baseline Keras model: 849709.00 bytes
Size of gzipped pruned and quantized TFlite model: 78743.00 bytes


In [235]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == labels_test).mean()
  return accuracy

In [236]:

interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9122978248745119
Pruned TF test accuracy: 0.9127160906791687
